In [1]:
import pandas as pd
import glob
import os
from Patient import Patient
from Chromosome import Chromosome
from ChrArm import ChrArm
from Cytoband import Cytoband
#from Segment import Segment
import time
import numpy as np

In [2]:
path_to_files = '../../HRD_score/data/allele_specific_cnv/allele_cnv_txt/'
path_to_formated_folder = '../../HRD_score/data/allele_specific_cnv/formatted_asc_files'

# Get all the files
files = glob.glob('../../HRD_score/data/allele_specific_cnv/allele_cnv_txt/*.seg.txt')
files = [os.path.basename(file) for file in files]

In [3]:
chromosome_data = pd.read_csv('../data/chromosome_data.csv', sep=',', header = 0, index_col = 0)
cytobands_data = pd.read_csv('../data/cytoband_UCSC_annotated.csv', sep=',', header = 0, index_col = 0)
HRD_results = pd.read_csv('../../HRD_score/data/HRD_scores_pan_cancer_annotated_v2.csv', sep=',', header = 0)
HRD_results_primary = HRD_results[HRD_results['Type'] == 'Primary']

In [ ]:
cytobands_data

In [ ]:
print(files[0])

In [4]:
test = pd.read_csv(path_to_files+files[0], sep='\t', header = 0)

# Function Segments

In [5]:
def filterout_sex_chromosome(data):
    '''
    Function to filter out the chrX and chrY from the dataframe
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    Output:
    data (dataframe): Allelic specific copy number dataframe without chrX and chrY
    '''
    return data[(data['Chromosome'] != 'chrY') & (data['Chromosome'] != 'chrX')]

def annotateSegmentStatus(data):
    '''
    Annotates the copy number dataframe with a new column called Status indicating if a segment is CN-AMP, CN-GAIN, CN-LOH, Homo-Del, Hemi-Del, Gain, Amp or Neutral
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    Output:
    data (dataframe): Allelic specific copy number dataframe with new column called Status
    '''
    data.loc[:, 'Status'] = 'Neutral'
    
    data.loc[(data['Copy_Number'] == 0), 'Status'] = 'Homo-Del'
    data.loc[(data['Copy_Number'] == 1), 'Status'] = 'Hemi-Del'
    
    data.loc[(data['Copy_Number'] == 3) & ((data['Major_Copy_Number'] > 0) & (data['Minor_Copy_Number'] > 0)), 'Status'] = 'Gain'
    data.loc[((data['Minor_Copy_Number'] == 1) & (data['Major_Copy_Number'] == 3)) |
             ((data['Minor_Copy_Number'] == 3) & (data['Major_Copy_Number'] == 1)) , 'Status'] = 'Gain'
    
    data.loc[(data['Copy_Number'] >= 5) & ((data['Minor_Copy_Number'] > 0) & (data['Major_Copy_Number'] > 0)), 'Status'] = 'Amp'
    
    
    
    data.loc[((data['Major_Copy_Number'] > 4) & (data['Minor_Copy_Number'] == 0)) |
             ((data['Minor_Copy_Number'] > 4) & (data['Major_Copy_Number'] == 0)), 'Status'] = 'CN-AMP'

    data.loc[((data['Major_Copy_Number'] == 0) & ((data['Minor_Copy_Number'] == 3) | (data['Minor_Copy_Number'] == 4))) |
             ((data['Minor_Copy_Number'] == 0) & ((data['Major_Copy_Number'] == 3) | (data['Major_Copy_Number'] == 4))), 'Status'] = 'CN-GAIN'
    
    data.loc[((data['Major_Copy_Number'] == 2) & (data['Minor_Copy_Number'] == 0)) |
             ((data['Major_Copy_Number'] == 0) & (data['Minor_Copy_Number'] == 2)), 'Status'] = 'CN-LOH'
    
    
    return data

def annotateSegmentLength(data):
    '''
    Adds the length of the segments to the copy number dataframe.
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    Output:
    data (dataframe): Allelic specific copy number dataframe with new column called Length
    '''
    data.loc[:, 'Length'] = data['End'] - data['Start']
    return data




# Functions for both Arm and Cytoband

In [6]:
def filter_by_chromosome(data, nr_chromosome):
    '''
    Extracts subdf of the data to only contain the given chromosome
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    nr_chromosome (string): chromosome number like 'chr1'
    Output:
    (dataframe): Filtered dataframe
    '''
    return data[data['Chromosome'] == nr_chromosome]

def filter_by_segments(chr_data, segment_indices):
    '''
    Extracts subdf of the chr_data (data only from one chromosome) to only contain the segments from the given index list.
    Input:
    chr_data (dataframe): Allelic specific copy number dataframe of one sample for one chromosome
    segment_indices (list of Int): List containing the indices of the segments that need to be extracted
    Output:
    (dataframe): Filtered dataframe
    '''
    return chr_data.loc[segment_indices]


def add_segment_info(chr_data, level_object):
    '''
    Selects the segments that are covering the given level (arm or cytoband). Adds a list of indices, fraction covered,
    segment length, copy numbers and type of events (Status) for these segments.
    Input:
    chr_data (dataframe): Allelic specific copy number dataframe of one sample for one chromosome
    level_object (ChrArm or Cytoband object): Level analysed
    Output:
    level_object (ChrArm or Cytoband object): Level object with added information
    '''
    
    ## Get variables from the object
    level_start = level_object.start
    level_end = level_object.end
    level_length = level_object.length
    
    ## Get the mask for the segments covering the level. There are 4 possible ways segments are on the level.
    ## Because each has their own calculation of the length of the segment 4 masks are made.
    ## maks_full: Segments that are in between the level; mask_end_in: Segments that end in the level but start before
    ## mask_start_in: Segments that start in the level but end after it
    ## mask_over: Segment that start befor and ends after the level
    mask_full = ((chr_data['Start'] >= level_start) & (chr_data['End'] <= level_end))
    mask_end_in = ((chr_data['End'] <= level_end) & (chr_data['End'] > level_start) & (chr_data['Start'] < level_start))
    mask_start_in = ((chr_data['Start'] >= level_start) & (chr_data['Start'] < level_end) & (chr_data['End'] > level_end) )
    mask_over = ((chr_data['Start'] <= level_start) & (chr_data['End'] >= level_end))
    
    row_indices_full = chr_data.index[mask_full].tolist()
    row_indices_end_in = chr_data.index[mask_end_in].tolist()
    row_indices_start_in = chr_data.index[mask_start_in].tolist()
    row_indices_over = chr_data.index[mask_over].tolist()
    
    fraction_covered = list()
    seg_lengths = list()
    events = list()
    copy_numbers = list()
    
    ## Subset the dataframe
    full = chr_data.loc[row_indices_full]
    end_in = chr_data.loc[row_indices_end_in]
    start_in = chr_data.loc[row_indices_start_in]
    over = chr_data.loc[row_indices_over]
    
    ## Segments inside the level, the length can just be taken
    for index, row in full.iterrows():
        length = row['Length']
        fc = length/level_length
        fraction_covered.append(fc)
        seg_lengths.append(length)
        events.append(row['Status'])
        copy_numbers.append(row['Copy_Number'])
    
    ## Segments that end in the level, subtract the start of the level from the end position of the segment for the length
    for index, row in end_in.iterrows():
        end = row['End']
        length = end - level_start
        fc = length/level_length
        fraction_covered.append(fc)
        seg_lengths.append(length)
        events.append(row['Status'])
        copy_numbers.append(row['Copy_Number'])
    
    ## Segments that start in the level, subtract the start point of the segment from the end of the level for the length
    for index, row in start_in.iterrows():
        start = row['Start']
        length = level_end - start
        fc = length/level_length
        fraction_covered.append(fc)
        seg_lengths.append(length)
        events.append(row['Status'])
        copy_numbers.append(row['Copy_Number'])
    
    ## Segments that start and end outside of the level covering it fully, take just the length of the level as segment length
    ## For mean length how to handle it ??
    for index, row in over.iterrows():
        fc = level_length/level_length
        fraction_covered.append(fc)
        seg_lengths.append(level_length)
        events.append(row['Status'])
        copy_numbers.append(row['Copy_Number'])
    
    row_indices = row_indices_full + row_indices_end_in + row_indices_start_in + row_indices_over
    
    level_object.segments_indices = row_indices
    level_object.segments_coverage = fraction_covered
    level_object.segments_length = seg_lengths
    level_object.segments_event = events
    level_object.segments_copy_number = copy_numbers
    
    if len(seg_lengths) == 0:
        level_object.mean_seg_length = 0.0
    else:
        level_object.mean_seg_length = np.average(seg_lengths)
    
    return level_object


def add_eventsInfo_and_cins(level_object, threshold = 90):
    '''
    Calculates the general and cn_cin for the given level object. Adds the mean length and number of specific events.
    Checks if the level has itself a specific event if at least threhold coverage is there.
    Input:
    level_object (ChrArm or Cytoband object)
    threshold (Int): Minimum coverage needed that the level is asigned a specific Status
    Output
    level_object (ChrArm or Cytoband object): With the added infromation and CINs
    '''
    
    level_events = level_object.segments_event
    
    events = list(np.unique(level_events))
    
    #if 'Neutral' in events:
    #    events.remove('Neutral')
    
    general_cin = 0
    cn_cin = 0
    
    
    for event in events:
        ## Get the indiexes for the list with the different informations
        indices = [i for i, value in enumerate(level_events) if value == event]
        mean_length = np.average([level_object.segments_length[i] for i in indices])
        number_of_event = len(indices)
        coverage = sum([level_object.segments_coverage[i] for i in indices]) * 100
        
        if coverage >= threshold:
            level_object.status = event
        
        ## Add depending on the event the needed information
        if event == 'CN-LOH':
            level_object.number_cn_loh = number_of_event
            level_object.mean_cn_loh_length = mean_length
            cn_cin = cn_cin + number_of_event
        if event == 'CN-GAIN':
            level_object.number_cn_gain = number_of_event
            level_object.mean_cn_gain_length = mean_length
            cn_cin = cn_cin + number_of_event
        if event == 'CN-AMP':
            level_object.number_cn_amp = number_of_event
            level_object.mean_cn_amp_length = mean_length
            cn_cin = cn_cin + number_of_event        
        
        if event == 'Homo-Del':
            level_object.number_homo_del = number_of_event
            level_object.mean_homo_del_length = mean_length
            general_cin = general_cin + number_of_event        
        if event == 'Hemi-Del':
            level_object.number_hemi_del = number_of_event
            level_object.mean_hemi_del_length = mean_length
            general_cin = general_cin + number_of_event
        if event == 'Neutral':
            level_object.number_neutral = number_of_event
            level_object.mean_neutral_length = mean_length
        if event == 'Gain':
            level_object.number_gain = number_of_event
            level_object.mean_gain_length = mean_length
            general_cin = general_cin + number_of_event        
        if event == 'Amp':
            level_object.number_amp = number_of_event
            level_object.mean_amp_length = mean_length
            general_cin = general_cin + number_of_event
        
    level_object.general_cin = general_cin
    level_object.cn_cin = cn_cin
    
    return level_object
    


def compute_average_copy_number(level_object):
    '''
    Computes the average copy number for a given level object. Sum of copy number times the coverage, divided by the total coverage of all segments.
    Input:
    level_object (ChrArm or Cytoband object)
    Output
    level_object (ChrArm or Cytoband object): With the added average copy number
    '''
    
    total_fc = sum(level_object.segments_coverage)
    
    total_cn = 0
    
    for index, cn in enumerate(level_object.segments_copy_number):
        fc = level_object.segments_coverage[index]
        
        total_cn = total_cn + (cn * fc)
    
    ## If there is no segments there is no average copy number
    if total_fc == 0.0:
        level_object.average_cn = float('NaN')
    else:
        level_object.average_cn = total_cn/total_fc
    
    return level_object



# Functions for Arms

In [7]:
def createBothArms(data, chromosome_data, nr_chromosome):
    '''
    Creates both arms of a given chromosome and adds/calculates all attributes
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    chromosome_data (dataframe): Dataframe containing information about the chromosomes such as arm length ect. (look at Chromosome_data.ipynb)
    nr_chromosome (string): chromosome number like 'chr1'
    Output:
    p_arm (ChrArm object): The smaller p arm of the chromosome
    q_arm (ChrArm object): The larger q arm of the chromosome
    '''
    
    # arm_p, arm_q = initiateBothChrArms(chromosome_data, nr_chromosome)
    
    ## Get the informations for the arm p and q
    p_start = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['start_Sarm_p'].values[0]
    p_end = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['end_Sarm_p'].values[0]
    p_length = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['length_Sarm_p'].values[0]
    
    q_start = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['start_Larm_q'].values[0]
    q_end = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['end_Larm_q'].values[0]
    q_length = chromosome_data[chromosome_data['Chromosome'] == nr_chromosome]['length_Larm_q'].values[0]
    
    ## Create two instances of the Arm object for arm p and q
    arm_p = ChrArm('p',nr_chromosome, p_start, p_end, p_length)
    arm_q = ChrArm('q', nr_chromosome, q_start, q_end, q_length)
    
    ## Filter the data for the chromosome
    chr_data = filter_by_chromosome(data, nr_chromosome)
    
    ## Add segment infos, event infos, general and cn CIN and the average copy number
    arm_p = add_segment_info(chr_data, arm_p)
    arm_q = add_segment_info(chr_data, arm_q)
    
    arm_p = add_eventsInfo_and_cins(arm_p, threshold = 90)
    arm_q = add_eventsInfo_and_cins(arm_q, threshold = 90)
    
    arm_p = compute_average_copy_number(arm_p)
    arm_q = compute_average_copy_number(arm_q)
    
    
    return arm_p, arm_q

In [108]:
test = pd.read_csv(path_to_files+files[300], sep='\t', header = 0)
test_2 = filterout_sex_chromosome(test)
test_2_2 = annotateSegmentLength(test_2.copy())
test_3 = annotateSegmentStatus(test_2_2.copy())

## Arm Test

In [9]:
arm_p, arm_q = createBothArms(test_3, chromosome_data, 'chr13')

In [ ]:
arm_p.printAttributes()

In [96]:
arm_p, arm_q = createBothArms(test_3, chromosome_data, 'chr12')
print('Arm p')
arm_p.printAttributes()
print('Arm q')
arm_q.printAttributes()

Arm p
Chr number
chr12
Chromosome arm combined name
chr12_p
Start
0
End
35500000
Lenght
35500001
Status
Neutral
General CIN
0
CN-CIN
3
Average Copy number
2.0
Number CN-LOH
3
Number CN-GAIN
0
Number CN-AMP
0
Number Homo-Del
0
Number Hemi-Del
0
Number Neutral
3
Number Gain
0
Number Amp
0
Mean segment length
5906196.166666667
Mean CN-LOH length
885416.0
Mean CN-GAIN length
0
Mean CN-AMP length
0
Mean Homo-Del length
0
Mean Hemi-Del length
0
Mean Neutral length
10926976.333333334
Mean Gain length
0
Mean Amp length
0
Indices Segments
[425, 426, 427, 428, 429, 430]
Coverage Segments
[0.3717738486824268, 0.012654309502695507, 0.427319480920578, 0.0034445914522650296, 0.12431309508977197, 0.05872498426126805]
Length segments
[13197972, 449228, 15169842, 122283, 4413115, 2084737]
Event segements
['Neutral', 'CN-LOH', 'Neutral', 'CN-LOH', 'Neutral', 'CN-LOH']
Segments copy number
[2, 2, 2, 2, 2, 2]
Arm q
Chr number
chr12
Chromosome arm combined name
chr12_q
Start
35500001
End
133275309
Lenght
9

In [97]:
chromosome = arm_p.length + arm_q.length

In [98]:
arm_p.length/chromosome

0.2663659252892822

In [99]:
filter_by_chromosome(test_3, 'chr13')

,GDC_Aliquot,Chromosome,Start,End,Copy_Number,Major_Copy_Number,Minor_Copy_Number,Length,Status
452,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,18452809,53697969,2,1,1,35245160,Neutral
453,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,53699083,54255309,3,2,1,556226,Gain
454,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,54260053,62714497,2,1,1,8454444,Neutral
455,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,62714803,62946161,2,2,0,231358,CN-LOH
456,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,62946416,63637726,2,1,1,691310,Neutral
457,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,63638572,64009866,2,2,0,371294,CN-LOH
458,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,64011676,69147789,2,1,1,5136113,Neutral
459,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,69147847,70085683,2,2,0,937836,CN-LOH
460,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,70087594,71279307,2,1,1,1191713,Neutral
461,fa6eb6bd-0761-458c-b19c-e9166eccbad0,chr13,71279591,71597717,2,2,0,318126,CN-LOH


# Functions Cytobands

In [100]:
def createAllCytoband_of_chr(data, cytoband_data, nr_chromosome):
    '''
    Creates all cytobands of one chromosome and adds/calculates all attributes
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    cytoband_data (dataframe): Cytoband info dataframe
    nr_chromosome (string): chromosome number like 'chr1'
    Output:
    cytoband_list (list of Cytoband objects)
    '''
    
    #cytoband_list = initiateAllCytobands(cytobands_data, nr_chromosome)
    
    ## Filter the data for the specific chromosome
    chr_data = filter_by_chromosome(data, nr_chromosome)
    
    cytoband_list = list()
    
    ## Get the numbers of the cytoband for that chromosome
    cytoband_data_chr = cytoband_data[cytoband_data['chromosome'] == nr_chromosome]
    cytoband_nr_list = list(cytoband_data_chr['cytoband'])
    
    for cyto_number in cytoband_nr_list:
        
        ## Get the information on the cytoband
        start = cytoband_data_chr[cytoband_data_chr['cytoband'] == cyto_number]['start'].values[0]
        end = cytoband_data_chr[cytoband_data_chr['cytoband'] == cyto_number]['end'].values[0]
        length = cytoband_data_chr[cytoband_data_chr['cytoband'] == cyto_number]['length'].values[0]
        
        ## Create an instance of a Cytoband object, add the segment infos, the event infos, the general and cn CIn, and the average copy number
        cytoband = Cytoband(cyto_number, nr_chromosome, start, end, length)
        cytoband = add_segment_info(chr_data, cytoband)
    
        cytoband = add_eventsInfo_and_cins(cytoband, threshold = 90)

        cytoband = compute_average_copy_number(cytoband)
        
        cytoband_list.append(cytoband)

    return cytoband_list

## Cytoband test

In [ ]:
test_full_list = createAllCytoband_of_chr(test_3, cytobands_data, 'chr1')

In [ ]:
test_full_list[0].printAttributes()

# Functions Chromosome

In [101]:
def add_attributes_values_chromosome(chr_data, chromosome, threshold):
    '''
    Fills all attributes of the given Chromosome object, such as mean of segments length (general and event specific),
    Computes the general CIN and cn_Cin. 
    Checks if the chromosome shows it self a specific event (if the coverage is bigger or equal the given threshold of a given event type)
    There is no need for checking the segments as all are on the chromsome.
    Input:
    chr_data (dataframe): Allelic specific copy number dataframe of one sample for one chromosome
    chromosome (Chromosome object)
    Output:
    chromosome (Chromosome object): With all attributes added
    '''
    
    chromosome_length = chromosome.length
    
    chromosome.mean_seg_length = np.average(chr_data['Length'])
    
    events = list(np.unique(chr_data['Status']))
    
    general_cin = 0
    cn_cin = 0
    
    for event in events:
        event_df = chr_data[chr_data['Status'] == event]
        
        mean_length = np.average(event_df['Length'])
        number_of_event = event_df.shape[0]
        
        coverage = (event_df['Length']/chromosome_length).sum() * 100
        
        #if coverage >= threshold and not event == 'Neutral':
        if coverage >= threshold:
            chromosome.status = event
        
        if event == 'CN-LOH':
            chromosome.number_cn_loh = number_of_event
            chromosome.mean_cn_loh_length = mean_length
            cn_cin = cn_cin + number_of_event
        if event == 'CN-GAIN':
            chromosome.number_cn_gain = number_of_event
            chromosome.mean_cn_gain_length = mean_length
            cn_cin = cn_cin + number_of_event
        if event == 'CN-AMP':
            chromosome.number_cn_amp = number_of_event
            chromosome.mean_cn_amp_length = mean_length
            cn_cin = cn_cin + number_of_event        
        
        if event == 'Homo-Del':
            chromosome.number_homo_del = number_of_event
            chromosome.mean_homo_del_length = mean_length
            general_cin = general_cin + number_of_event        
        if event == 'Hemi-Del':
            chromosome.number_hemi_del = number_of_event
            chromosome.mean_hemi_del_length = mean_length
            general_cin = general_cin + number_of_event        
        if event == 'Gain':
            chromosome.number_gain = number_of_event
            chromosome.mean_gain_length = mean_length
            general_cin = general_cin + number_of_event        
        if event == 'Amp':
            chromosome.number_amp = number_of_event
            chromosome.mean_amp_length = mean_length
            general_cin = general_cin + number_of_event
        if event == 'Neutral':
            chromosome.number_neutral = number_of_event
            chromosome.mean_neutral_length = mean_length
    
    
    chromosome.general_cin = general_cin
    chromosome.cn_cin = cn_cin
    
    ## Calculation of the average copy number (same calculation as for Arm or Cytoband)
    coverages = list(chr_data['Length']/chromosome_length)
    
    cns = list(chr_data['Copy_Number'])
    total_cn = sum([cn * fc for cn, fc in zip(cns, coverages)])
    
    total_fc = sum(coverages)
    
    chromosome.average_cn = total_cn/total_fc

    return chromosome




def createWholeChromosome(data, chromosome_data, mode, cytoband_data = None):
    '''
    Creates all chromosome and adds/calculates all attributes
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    chromosome_data (dataframe): Dataframe containing information about the chromosomes such as arm length ect. (look at Chromosome_data.ipynb)
    mode (string): Indicates what level should be calculated ('Chromosome', 'Cytoband', 'Arm', 'All')
    cytoband_data (dataframe): Cytoband info dataframe, default is None, so that if mode is not Cytoband or All, not input is needed.
    Output:
    chromosome_list (list of Chromosome objects)
    '''
    
    #chromosome_list = initiateAllChromosome(chromosome_data)
    
    #for i in range(0,len(chromosome_list)):
        #chromosome = chromosome_list[i]
        
    chromosome_list = list()
    
    chromosome_nr_list = list(chromosome_data['Chromosome'])
    
    for chr_nr in chromosome_nr_list:
        
        length = chromosome_data[chromosome_data['Chromosome'] == chr_nr]['length'].values[0]
        
        chromosome = Chromosome(chr_nr, length)
        
        #chromosome_list.append(chromosome)
        
        chr_data = filter_by_chromosome(data, chromosome.chromosome_number)
        
        chromosome = add_attributes_values_chromosome(chr_data, chromosome, threshold = 90)
        
        ## Depending on the mode different levels are analysed and generated
        if mode == 'Arm':
            
            arm_p, arm_q = createBothArms(data, chromosome_data, chromosome.chromosome_number)
            chromosome.arm_p = arm_p
            chromosome.arm_q = arm_q
            
        elif mode == 'Cytoband':
            
            cytoband_list = createAllCytoband_of_chr(data, cytoband_data, chromosome.chromosome_number)
            chromosome.cytobands = cytoband_list
            
        elif mode == 'All':
            
            arm_p, arm_q = createBothArms(data, chromosome_data, chromosome.chromosome_number)
            chromosome.arm_p = arm_p
            chromosome.arm_q = arm_q
            
            cytoband_list = createAllCytoband_of_chr(data, cytoband_data, chromosome.chromosome_number)
            chromosome.cytobands = cytoband_list
            
            
        #chromosome_list[i] = chromosome
        chromosome_list.append(chromosome)

    return chromosome_list


## Test Chromosome

In [109]:
chromosome_data = filterout_sex_chromosome(chromosome_data)
test_list_chromosome = createWholeChromosome(test_3, chromosome_data, mode = 'Arm' )

In [ ]:
filter_by_chromosome(test_3, 'chr1')

In [110]:
for chromosome in test_list_chromosome:
    arm_p = chromosome.arm_p
    arm_q = chromosome.arm_q
    length = chromosome.length
    chr_data = filter_by_chromosome(test_3, chromosome.chromosome_number) 
    print(chromosome.chromosome_number)
    print('Length')
    print(length/1000000)
    print('Of total length arm_p covers')
    print(arm_p.length/length)
    print('Of total length arm_q covers')
    print(arm_q.length/length)
    print('Number of segments')
    print(chr_data.shape[0])
    print('Number on arm p')
    print(len(arm_p.segments_indices))
    print('Number on arm q')
    print(len(arm_q.segments_indices))
    if chromosome.mean_seg_length > arm_p.length:
        print('Mean length is bigger')
    else:
        print('Mean length is smaller')

chr1
Length
248.956423
Of total length arm_p covers
0.4956690794035067
Of total length arm_q covers
0.5043309165797261
Number of segments
11
Number on arm p
1
Number on arm q
10
Mean length is smaller
chr10
Length
133.797423
Of total length arm_p covers
0.2974646305407541
Of total length arm_q covers
0.7025353619852603
Number of segments
8
Number on arm p
4
Number on arm q
5
Mean length is smaller
chr11
Length
135.086623
Of total length arm_p covers
0.3953019167560359
Of total length arm_q covers
0.6046980758413066
Number of segments
5
Number on arm p
3
Number on arm q
3
Mean length is smaller
chr12
Length
133.27531
Of total length arm_p covers
0.266365923290668
Of total length arm_q covers
0.7336340692060668
Number of segments
1
Number on arm p
1
Number on arm q
1
Mean length is bigger
chr13
Length
114.364329
Of total length arm_p covers
0.1547685467555185
Of total length arm_q covers
0.8452314445004963
Number of segments
82
Number on arm p
0
Number on arm q
82
Mean length is smaller


# Function Patient

In [11]:
def createAllPatients(files_list, chromosome_data, mode, path_to_files, cytoband_data = None):
    '''
    Creates a patient object for every sample file and add all attributes and the lower levels
    Input:
    files_list (list of strings): List containing the file names
    patient_list (list of Patient objects)
    chromosome_data (dataframe): Dataframe containing information about the chromosomes such as arm length ect. (look at Chromosome_data.ipynb)
    mode (string): Indicates what level should be calculated ('Chromosome', 'Cytoband', 'Arm', 'All')
    path_to_files (string): Path to the folder with the files in it
    cytoband_data (dataframe): Cytoband info dataframe, default is None, so that if mode is not Cytoband or All, not input is needed.
    Output:
    patient_list (Patient objects)
    '''
    patient_list = list()
    
    nr_files = len(files_list)
    
    i = 0
    
    for file in files_list:
        
        patient = Patient(file)
        
        data = loadPatientData(path_to_files, file)
        
        data = preparePatientData(data)
        
        chromosome_data = filterout_sex_chromosome(chromosome_data)
        
        chromosome_list = createWholeChromosome(data, chromosome_data, mode, cytoband_data)
        
        patient.chromosomes = chromosome_list
        
        patient = add_attributes_to_patient(patient)
        
        
        patient_list.append(patient)
        
        i = i+1
        
        if i%100 == 0:
            print('Progress: '+str(i)+'/'+str(nr_files)+ ' patients done')
        
    return patient_list



def add_attributes_to_patient(patient):
    '''
    Calculates the number of amplifications, deletions and the CIN for a given patient and adds it to the object.
    Additionally it adds the number of copy neutral events to the patient.
    Input:
    patient (Patient object)
    Output
    patient (Patient object): With the added amp, del, cin, n_cn_amp, n_cn_gain and n_cn_loh
    '''
    
    chr_list = patient.chromosomes
    
    patient.general_cin = sum(chromosome.general_cin for chromosome in chr_list)
    patient.cn_cin = sum(chromosome.cn_cin for chromosome in chr_list)
    
    patient.number_cn_amp = sum(chromosome.number_cn_amp for chromosome in chr_list)
    patient.number_cn_gain = sum(chromosome.number_cn_gain for chromosome in chr_list)
    patient.number_cn_loh = sum(chromosome.number_cn_loh for chromosome in chr_list)
    patient.number_homo_del = sum(chromosome.number_homo_del for chromosome in chr_list)
    patient.number_hemi_del = sum(chromosome.number_hemi_del for chromosome in chr_list)
    patient.number_neutral = sum(chromosome.number_neutral for chromosome in chr_list)
    patient.number_gain = sum(chromosome.number_gain for chromosome in chr_list)
    patient.number_amp = sum(chromosome.number_amp for chromosome in chr_list)
    
    
    return patient

def loadPatientData(path_to_files, filename):
    '''
    Loads the allelic specific copy number data for a given patient
    Input:
    path_to_files (string): Path to the folder with the files in it
    filename (string): Allelic specific copy number file name
    Output:
    (dataframe): Allelic specific copy number dataframe of one sample
    '''
    return pd.read_csv(path_to_files+filename, sep='\t', header = 0)

def preparePatientData(data):
    '''
    Filters out the sex chromosomes from the dataframe and adds the length of the segement and the Copy neutral state information
    Input:
    data (dataframe): Allelic specific copy number dataframe of one sample
    Output:
    data (dataframe): Input data, filtered and with added columns
    '''
    data_filtered = filterout_sex_chromosome(data)
    data_length = annotateSegmentLength(data_filtered.copy())
    data_cn = annotateSegmentStatus(data_length.copy())
    return data_cn


def addHRDResultsToPatient(patient_list, HRD_results):
    '''
    Adds the HRD results to each patient
    Input
    patient_list (list of Patient objects)
    HRD_results (dataframe): Contains the HRD scores
    Output:
    patient_list (list of Patient objects)
    '''
    for i in range(0,len(patient_list)):
        patient = patient_list[i]
        hrd = HRD_results[HRD_results['File Name'] == patient.name]['HRD_sum'].values[0]
        lst = HRD_results[HRD_results['File Name'] == patient.name]['LST'].values[0]
        loh = HRD_results[HRD_results['File Name'] == patient.name]['LOH'].values[0]
        tai = HRD_results[HRD_results['File Name'] == patient.name]['TAI'].values[0]
        
        patient.HRD = hrd
        patient.LOH = loh
        patient.LST = lst
        patient.TAI = tai
        
        patient_list[i] = patient
        
    return patient_list

## Test patient

In [ ]:
test_list = files[0:4]
mode = 'All'
patient_list_test = createAllPatients(test_list, chromosome_data, mode, path_to_files, cytobands_data)

In [ ]:
patient_list_test[0].printAttributes()

# General functions

In [12]:
def loadData(chromosome_data_path, cytoband_data_path, HRD_results_path):
    '''
    Load needed data such as chromosome data, cytoband data, and the HRD results (only using primary samples)
    Input:
    chromosome_data_path (string): Path to chromosome data
    cytoband_data_path (string): Path to cytoband data
    HRD_results_path (string): Path to HRD results
    Output:
    chromosome_data (dataframe)
    cytobands_data (dataframe)
    HRD_results_primary (dataframe)
    '''
    chromosome_data = pd.read_csv(chromosome_data_path, sep=',', header = 0, index_col = 0)
    cytobands_data = pd.read_csv(cytoband_data_path, sep=',', header = 0, index_col = 0)
    HRD_results = pd.read_csv(HRD_results_path, sep=',', header = 0)
    HRD_results_primary = HRD_results[HRD_results['Type'] == 'Primary']
    
    return chromosome_data, cytobands_data, HRD_results_primary



def writeResults(patient_list, project_id, mode, output_path):
    '''
    Writes the results into tables. For each level (Patient, Chromosome, Arm, Cytoband) a table is created, which depends on the mode.
    Input:
    patient_list (list of Patients objects)
    project_id (string): Cancer type/ Project ID in the from of 'TCGA-LUAD'
    output_path (string): Path to the result folder
    '''
    
    chromosome_data = []
    arm_data = []
    cytoband_data = []
    
    attributes = ['length', 'general_cin', 'cn_cin', 'status', 'average_cn',
                  'number_cn_loh', 'number_cn_gain', 'number_cn_amp',
                 'number_homo_del', 'number_hemi_del', 'number_neutral',
                 'number_gain', 'number_amp',
                  'mean_seg_length', 'mean_cn_loh_length', 'mean_cn_gain_length',
                 'mean_cn_amp_length', 'mean_homo_del_length', 'mean_hemi_del_length',
                 'mean_neutral_length', 'mean_gain_length', 'mean_amp_length']
    
    df_patient = pd.DataFrame(columns=['File Name',
                                       'general_cin',
                                       'cn_cin',
                                       'n_cn_loh',
                                       'n_cn_gain',
                                       'n_cn_amp',
                                       'n_homo_del',
                                       'n_hemi_del',
                                       'n_neutral',
                                       'n_gain',
                                       'n_amp',
                                       'HRD_sum',
                                       'LST',
                                       'LOH',
                                       'TAI'])
    
    for patient in patient_list:
        ## Add new patient
        df_patient.loc[len(df_patient)] = [patient.name,
                                           patient.general_cin,
                                           patient.cn_cin,
                                           patient.number_cn_loh,
                                           patient.number_cn_gain,
                                           patient.number_cn_amp,
                                           patient.number_homo_del,
                                           patient.number_hemi_del,
                                           patient.number_neutral,
                                           patient.number_gain,
                                           patient.number_amp,
                                           patient.HRD,
                                           patient.LST,
                                           patient.LOH,
                                           patient.TAI]
        
        
        patient_data_chromosome = {'File Name': patient.name}
        
        ## Create dicts depending on the mode
        if mode == 'Arm':
            patient_data_arm = {'File Name': patient.name}
        elif mode == 'Cytoband':
            patient_data_cytoband = {'File Name': patient.name}
        elif mode == 'All':
            patient_data_arm = {'File name': patient.name}
            patient_data_cytoband = {'File Name': patient.name}
            

        for chromosome in patient.chromosomes:
            
            ## Add chromosome results
            for attribute in attributes:
                column_name = f"{chromosome.chromosome_number}_{attribute}"
                patient_data_chromosome[column_name] = getattr(chromosome, attribute)
            
            ## Add arm results
            if mode == 'Arm':
                for arm_label in ['p', 'q']:
                    arm = getattr(chromosome, f"arm_{arm_label}")
            
                    for attribute in attributes:
                        column_name_arm = f"{arm.chromosome_arm}_{attribute}"
                        patient_data_arm[column_name_arm] = getattr(arm, attribute)
                        
            ## Add cytoband results            
            elif mode == 'Cytoband':
                
                for cytoband in chromosome.cytobands:
                    for attribute in attributes:
                        column_name_cytoband = f"{cytoband.chromosome_cyto}_{attribute}"
                        patient_data_cytoband[column_name_cytoband] = getattr(cytoband, attribute)
            ## Add arm and cytoband results
            elif mode == 'All':
                
                for arm_label in ['p', 'q']:
                    arm = getattr(chromosome, f"arm_{arm_label}")
            
                    for attribute in attributes:
                        column_name_arm = f"{arm.chromosome_arm}_{attribute}"
                        patient_data_arm[column_name_arm] = getattr(arm, attribute)
                        
                for cytoband in chromosome.cytobands:
                    for attribute in attributes:
                        column_name_cytoband = f"{cytoband.chromosome_cyto}_{attribute}"
                        patient_data_cytoband[column_name_cytoband] = getattr(cytoband, attribute)
                        
        chromosome_data.append(patient_data_chromosome)
        if mode == 'Arm':
            arm_data.append(patient_data_arm)
        elif mode == 'Cytoband':
            cytoband_data.append(patient_data_cytoband)
        elif mode == 'All':
            arm_data.append(patient_data_arm)
            cytoband_data.append(patient_data_cytoband)
    
    ## Create the dataframes and save them
    df_chr = pd.DataFrame(chromosome_data)
    
    df_patient.to_csv(output_path+'/'+project_id+'_level_patient.csv', sep=',', header = True)
    df_chr.to_csv(output_path+'/'+project_id+'_level_chromosome.csv', sep=',', header = True)
    
    if mode == 'Arm':
        
        df_arm = pd.DataFrame(arm_data)
        df_arm.to_csv(output_path+'/'+project_id+'_level_arms.csv', sep=',', header = True)
        
    elif mode == 'Cytoband':
        
        df_cyto = pd.DataFrame(cytoband_data)
        df_cyto.to_csv(output_path+project_id+'_level_cytobands.csv', sep=',', header = True)
        
    elif mode == 'All':
        
        df_arm = pd.DataFrame(arm_data)
        df_arm.to_csv(output_path+'/'+project_id+'_level_arms.csv', sep=',', header = True)
        df_cyto = pd.DataFrame(cytoband_data)
        df_cyto.to_csv(output_path+'/'+project_id+'_level_cytobands.csv', sep=',', header = True)
        

def checkFolder(path):
    '''
    Checks if directory exits
    Input
    path (string)
    Output:
    (boolean)
    '''
    return (os.path.exists(path) and os.path.isdir(path))

def run_CIN_pipeline(path_to_files, chromosome_data_path, cytoband_data_path, HRD_results_path, mode, output_path, version, selection = []):
    
    '''
    Function to run the pipeline.
    Input:
    path_to_files (string): Path to the folder with the allelic specific copy number files
    chromosome_data_path (string): Path to chromosome data
    cytoband_data_path (string): Path to cytoband data
    HRD_results_path (string): Path to HRD results
    mode (string): Indicates for which levels results should be computed. Possible mode are 'Chromosome', 'Arm', 'Cytoband', 'All'.
                    Every level includes Patient and Chromosome output ('Chromosome' is for only Patient + Chromosome level)
    output_path (string): Path to the folder at which location the result folder is created
    version (string): Indicates the run, if the same version is run twice it check for each cancertype if there are results and if there are it skips it.
    selection (list of strings): Default is empty, if given a list of project IDs (like 'TCGA-LUAD') only the selected cohort are computed
    '''
    
    ## Load data
    chromosome_data, cytoband_data, HRD_results = loadData(chromosome_data_path, cytoband_data_path, HRD_results_path)
    
    ## Get Project IDs
    cancertypes = np.unique(HRD_results['Project ID'])
    
    ## Path to result folder
    mainresults_path = output_path + '/CIN_Output_version_'+ version
    
    ## Creates the results folder if not there
    if not checkFolder(mainresults_path):
        os.makedirs(mainresults_path)
    
    ## If only selection of cohort should be run, change the cancertype list to the selection
    if not len(selection) == 0:
        cancertypes = selection
    
    ## Dict to save the times need to run a cohort
    time_dict = dict()
    
    for cancertype in cancertypes:
        
        ## Creates the results subfolder for a cancertype if there is non
        ## If there is one and files are in there, the cancertype is skipped
        output_path_cancertype = mainresults_path + '/' + cancertype
        if checkFolder(output_path_cancertype):
            if len(os.listdir(output_path_cancertype)) > 0:
                print(cancertype + ' already computed')
                continue
        else:
            os.makedirs(output_path_cancertype)
        
        ## Start timer
        start_time = time.time()
        
        print('Start of cohort '+ cancertype)
        
        ## Subset HRD results
        HRD_results_sub = HRD_results[HRD_results['Project ID'] == cancertype]
        
        ## Get the file names of the samples
        files = list(HRD_results_sub['File Name'])
        
        ## Create all patients object for each sample (main computation)
        patient_list = createAllPatients(files, chromosome_data, mode, path_to_files, cytoband_data)
        
        ## Add HRD results
        patient_list = addHRDResultsToPatient(patient_list, HRD_results_sub)
        
        print('Finished computation, start with writing the results into tables')
        
        ## Wrtie the results to tables
        writeResults(patient_list, cancertype , mode, output_path_cancertype)
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        ## Save needed time
        time_dict[cancertype] = execution_time
        
        print('Finished cohort '+ cancertype +' in '+str(execution_time))
    
    time_df = pd.DataFrame.from_dict(time_dict, orient='index', columns=['Time in s'])
    
    ### Rethink this
    ## Check if time already has been measured for this version, and if that is so, add new and old times together
    if os.path.exists(mainresults_path+'/timetable_'+version+'.csv'):
        old_results = pd.read_csv(mainresults_path+'/timetable_'+version+'.csv',index_col = 0)
        merged_times = pd.concat([old_results, time_df])
        merged_times.to_csv(mainresults_path+'/timetable_'+version+'.csv', sep=',', header = True)
    else:
        time_df.to_csv(mainresults_path+'/timetable_'+version+'.csv', sep=',', header = True)

In [13]:
chromosome_data_path = '../data/chromosome_data.csv'
cytoband_data_path = '../data/cytoband_UCSC_annotated.csv'
HRD_results_path = '../../HRD_score/data/HRD_scores_pan_cancer_annotated_v2.csv'

## Test general functions

In [14]:
run_CIN_pipeline( '../../HRD_score/data/allele_specific_cnv/allele_cnv_txt/', chromosome_data_path, cytoband_data_path, HRD_results_path, mode = 'All', output_path = '../data', version = '2_0')

Start of cohort TARGET-ALL-P2
Progress: 100/293 patients done
Progress: 200/293 patients done
Finished computation, start with writing the results into tables
Finished cohort TARGET-ALL-P2 in 842.6975343227386
Start of cohort TARGET-AML
Finished computation, start with writing the results into tables
Finished cohort TARGET-AML in 195.06833481788635
Start of cohort TARGET-CCSK
Finished computation, start with writing the results into tables
Finished cohort TARGET-CCSK in 32.1599235534668
Start of cohort TARGET-OS
Finished computation, start with writing the results into tables
Finished cohort TARGET-OS in 238.9998960494995
Start of cohort TCGA-ACC
Finished computation, start with writing the results into tables
Finished cohort TCGA-ACC in 260.76837515830994
Start of cohort TCGA-BLCA
Progress: 100/391 patients done
Progress: 200/391 patients done
Progress: 300/391 patients done
Finished computation, start with writing the results into tables
Finished cohort TCGA-BLCA in 1135.434262752533